# Resilient Distributed Datasets in Spark

It turns out DataFrames are implemented on top of RDDs.

- dataframes have schema
- RDD is simply a collection of objects

## Map and reduce



In [1]:
from pathlib import Path

from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/23 15:10:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('../data/taxi_ingest_data/parts/green/*/*')

`.rdd` returns the underlying RDD

In [5]:
df_green.rdd.take(3)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 12, 33, 7), lpep_dropoff_datetime=datetime.datetime(2020, 1, 23, 12, 38, 42), store_and_fwd_flag='N', RatecodeID=1, PULocationID=24, DOLocationID=41, passenger_count=1, trip_distance=0.9300000071525574, fare_amount=6.0, extra=0.0, mta_tax=0.5, tip_amount=1.3600000143051147, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.30000001192092896, total_amount=8.15999984741211, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 0, 7, 7), lpep_dropoff_datetime=datetime.datetime(2020, 1, 23, 0, 9, 47), store_and_fwd_flag='N', RatecodeID=1, PULocationID=193, DOLocationID=193, passenger_count=1, trip_distance=0.4699999988079071, fare_amount=4.0, extra=0.5, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.30000001192092896, total_amount=5.300000190734863, payment_type=2, trip_type=1, congestion_surcharge=0.0),


The `Row` object is also returned if we `.take` from the DataFrame

In [6]:
df_green.take(3)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 12, 33, 7), lpep_dropoff_datetime=datetime.datetime(2020, 1, 23, 12, 38, 42), store_and_fwd_flag='N', RatecodeID=1, PULocationID=24, DOLocationID=41, passenger_count=1, trip_distance=0.9300000071525574, fare_amount=6.0, extra=0.0, mta_tax=0.5, tip_amount=1.3600000143051147, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.30000001192092896, total_amount=8.15999984741211, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 0, 7, 7), lpep_dropoff_datetime=datetime.datetime(2020, 1, 23, 0, 9, 47), store_and_fwd_flag='N', RatecodeID=1, PULocationID=193, DOLocationID=193, passenger_count=1, trip_distance=0.4699999988079071, fare_amount=4.0, extra=0.5, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.30000001192092896, total_amount=5.300000190734863, payment_type=2, trip_type=1, congestion_surcharge=0.0),


In [12]:
cols = ['lpep_pickup_datetime', 'PULocationID', 'total_amount']
rdd_green = df_green \
                .select(*cols) \
                .rdd

In [9]:
from datetime import datetime

In [14]:
start = datetime(year=2020, month=1, day=1)
def filter_datetime(row):
    """
    pyspark.RDD.filter callable
    only accepts row as argument
    """
    return row.lpep_pickup_datetime >= start

In [17]:
def prepare_for_groupby(row):
    # assigning our keys
    # truncate to datetime to hour
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    # key becomes a tuple
    key = (hour, zone)

    # assigning values 
    amount = row.total_amount
    count = 1
    # value's also a tuple so that we have a 1:1 key: value relation
    value = (amount, count)
    return (key, value)

In [18]:
rdd_green \
    .filter(filter_datetime) \
    .map(prepare_for_groupby) \
    .take(10)

[((datetime.datetime(2020, 1, 23, 12, 0), 24), (8.15999984741211, 1)),
 ((datetime.datetime(2020, 1, 23, 0, 0), 193), (5.300000190734863, 1)),
 ((datetime.datetime(2020, 1, 29, 12, 0), 33), (23.979999542236328, 1)),
 ((datetime.datetime(2020, 1, 3, 15, 0), 224), (12.569999694824219, 1)),
 ((datetime.datetime(2020, 1, 10, 6, 0), 74), (10.5600004196167, 1)),
 ((datetime.datetime(2020, 1, 27, 18, 0), 226), (20.299999237060547, 1)),
 ((datetime.datetime(2020, 1, 27, 7, 0), 74), (35.689998626708984, 1)),
 ((datetime.datetime(2020, 1, 3, 16, 0), 75), (8.760000228881836, 1)),
 ((datetime.datetime(2020, 1, 4, 19, 0), 129), (36.060001373291016, 1)),
 ((datetime.datetime(2020, 1, 6, 15, 0), 61), (4.300000190734863, 1))]